In [13]:
import mysql.connector

def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="AI"
    )

In [14]:
def balance_agent(state):
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)
    cursor.execute("SELECT balance FROM customers WHERE account_no = %s", ("1234",))
    result = cursor.fetchone()
    conn.close()
    return {**state, "result": f"Your current balance is ₹{result['balance']}."}

def transfer_agent(state):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("UPDATE customers SET balance = balance - 5000 WHERE account_no = %s", ("1234",))
    cursor.execute("INSERT INTO transactions (account_no, amount, type) VALUES (%s, %s, %s)", ("1234", 5000, "debit"))
    conn.commit()
    conn.close()
    return {**state, "result": "₹5,000 has been transferred to account 1234."}

def statement_agent(state):
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)
    cursor.execute("SELECT amount, type, txn_date FROM transactions WHERE account_no = %s ORDER BY txn_date DESC LIMIT 3", ("1234",))
    rows = cursor.fetchall()
    conn.close()
    txn_summary = "; ".join([f"{row['type']} {row['amount']} on {row['txn_date']}" for row in rows])
    return {**state, "result": f"Last 3 transactions: {txn_summary}"}

def loan_agent(state):
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)
    cursor.execute("SELECT loan_amount, interest_rate FROM customers WHERE account_no = %s", ("1234",))
    result = cursor.fetchone()
    conn.close()
    return {**state, "result": f"Loan eligibility: ₹{result['loan_amount']} at {result['interest_rate']}% interest."}

def chat_agent(state):
    return {**state, "result": "Bank Assistant: How may I help you further?"}

In [15]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

class BankState(TypedDict):
    query: str
    intent: str
    result: str

def detect_intent(state: BankState) -> BankState:
    q = state["query"].lower()
    if "balance" in q:
        intent = "BALANCE_AGENT"
    elif "transfer" in q:
        intent = "TRANSFER_AGENT"
    elif "statement" in q or "transactions" in q:
        intent = "STATEMENT_AGENT"
    elif "loan" in q:
        intent = "LOAN_AGENT"
    else:
        intent = "CHAT_AGENT"
    return {**state, "intent": intent}

builder = StateGraph(BankState)
builder.add_node("detect", detect_intent)
builder.add_node("BALANCE_AGENT", balance_agent)
builder.add_node("TRANSFER_AGENT", transfer_agent)
builder.add_node("STATEMENT_AGENT", statement_agent)
builder.add_node("LOAN_AGENT", loan_agent)
builder.add_node("CHAT_AGENT", chat_agent)

builder.add_edge(START, "detect")
builder.add_conditional_edges("detect", lambda s: s["intent"], {
    "BALANCE_AGENT": "BALANCE_AGENT",
    "TRANSFER_AGENT": "TRANSFER_AGENT",
    "STATEMENT_AGENT": "STATEMENT_AGENT",
    "LOAN_AGENT": "LOAN_AGENT",
    "CHAT_AGENT": "CHAT_AGENT"
})
builder.add_edge("BALANCE_AGENT", END)
builder.add_edge("TRANSFER_AGENT", END)
builder.add_edge("STATEMENT_AGENT", END)
builder.add_edge("LOAN_AGENT", END)
builder.add_edge("CHAT_AGENT", END)

graph = builder.compile()
#display(Image(graph.get_graph().draw_mermaid_png()))

In [16]:
queries = [
    "Check my balance",
    "Transfer 5000 to account 1234",
    "Show my last transactions",
    "What is my loan eligibility?"
]

for q in queries:
    result = graph.invoke({"query": q, "intent": "", "result": ""})
    print(f"Query: {q}")
    print(f"Intent: {result['intent']}")
    print(f"Result: {result['result']}\n")

Query: Check my balance
Intent: BALANCE_AGENT
Result: Your current balance is ₹75000.00.

Query: Transfer 5000 to account 1234
Intent: TRANSFER_AGENT
Result: ₹5,000 has been transferred to account 1234.

Query: Show my last transactions
Intent: STATEMENT_AGENT
Result: Last 3 transactions: debit 5000.00 on 2025-10-12 11:25:17; debit 5000.00 on 2025-10-12 11:05:46; credit 2000.00 on 2025-10-12 11:05:46

Query: What is my loan eligibility?
Intent: LOAN_AGENT
Result: Loan eligibility: ₹1000000.00 at 8.00% interest.

